In [41]:
%use coroutines

In [42]:
import kotlinx.coroutines.channels.Channel
import kotlinx.coroutines.delay
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking

runBlocking {
    val channel = Channel<Int>(1)
    val outputChannel = Channel<Int>(1)

    val inputJob = launch {
        println("starting producer")
//        delay(200)
        (1..10).forEach {
            println("sending $it")
            channel.send(it)
            println("sent $it")
        }
    }

    val processor =launch {
        println("launching")
        for(it in channel) {
//            delay(100)
            outputChannel.send(it)
        }
    }

    launch {
        println("final")
        for(it in outputChannel) {
            delay(100)
            println("processed $it")
            if(it == 2) {
                println("Cancelling")
                inputJob.cancel()
                processor.cancel()
                cancel()
                return@launch
            }
        }
    }


}
println("done blocking")

starting producer
sending 1
sent 1
sending 2
launching
final
sent 2
sending 3
sent 3
sending 4
sent 4
sending 5
processed 1
sent 5
sending 6
processed 2
Cancelling
done blocking


# Use a pool of workers to handle events, but publish them in order

When we have an event we publish it along with a Deferred to two channels:
- we hash the key and send it to the worker that corresponds to the hash, it will complete the Deferred when it is done
- we also publish the Deferred to a separate channel that is a proxy for the Kafka publisher, it will await the completion of event processing 

This means we maintain the original order, but allow the work to be fanned out to multiple workers in a consistent way.

In [45]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.Channel
import kotlin.random.Random

data class Event(val key: String, val value: String)
data class EventResult(val event: Event, val processedValue: String)

fun CoroutineScope.launchEventWorker(workerId: Int, channel: Channel<Pair<Event, CompletableDeferred<EventResult>>>) =
    launch {
        for ((event, completion) in channel) {
            // random delay to simulate work
            delay(Random.nextLong(2_000L))
            val result = EventResult(event, "Processed by $workerId: ${event.value}")
            completion.complete(result)
            println("marked as completed: $result")
        }
    }

fun CoroutineScope.launchKafkaPublisher(channel: Channel<CompletableDeferred<EventResult>>) =
    launch {
        for (completion in channel) {
            val result = completion.await()
            println("Finished: $result")
        }
    }

fun main() = runBlocking {
    val numberOfWorkers = 4 // Number of workers
    val workersChannels = List(numberOfWorkers) { Channel<Pair<Event, CompletableDeferred<EventResult>>>(2) }
    val kafkaSendChannel = Channel<CompletableDeferred<EventResult>>(100)

    workersChannels.forEachIndexed { index, channel ->
        launchEventWorker(index, channel)
    }

    launchKafkaPublisher(kafkaSendChannel)
    
    // emulate kafka consumer which is getting via polling and will send values to the appropriate worker
    repeat(10) { i ->
        // dummy event with a key/value that shows what offset we're at
        val eventData = Event("key$i", "value$i")
        
        val deferred = CompletableDeferred<EventResult>()
        
        // spread the work across the pool of workers 
        val workerIndex = i % numberOfWorkers
        
        // send the event and the deferred to the worker for it to complete
        workersChannels[workerIndex].send(eventData to deferred)
        
        // also send the deferred to the kafka publisher, we're maintaining the original order of events
        // it will wait for the deferred to be completed by a worker
        kafkaSendChannel.send(deferred)
    }
    
    delay(10_000)
    println("Done")
    
    // Cleanup: close channels, etc.
    workersChannels.forEach { it.close() }
    kafkaSendChannel.close()
}


println("Notice that the Finished messages are in the same order as the orginal events\n")
main()


Notice that the Finished messages are in the same order as the orginal events
marked as completed: EventResult(event=Event(key=key2, value=value2), processedValue=Processed by 2: value2)
marked as completed: EventResult(event=Event(key=key1, value=value1), processedValue=Processed by 1: value1)
marked as completed: EventResult(event=Event(key=key0, value=value0), processedValue=Processed by 0: value0)
Finished: EventResult(event=Event(key=key0, value=value0), processedValue=Processed by 0: value0)
Finished: EventResult(event=Event(key=key1, value=value1), processedValue=Processed by 1: value1)
Finished: EventResult(event=Event(key=key2, value=value2), processedValue=Processed by 2: value2)
marked as completed: EventResult(event=Event(key=key3, value=value3), processedValue=Processed by 3: value3)
Finished: EventResult(event=Event(key=key3, value=value3), processedValue=Processed by 3: value3)
marked as completed: EventResult(event=Event(key=key6, value=value6), processedValue=Processed

true

### This pattern can be extended for acknowledging the kafka publish

Could also use a ticker rendesvous channel to do the committing of offsets back to the kafka brokers